In [ ]:
import coverage

In [ ]:
!coverage run -m pytest test_main.py
!coverage report -m

In [ ]:
cov = coverage.Coverage()
cov.load()
cov.analysis('main.py')[2]

In [ ]:
code = {}
with open('main.py') as f:
    for i, line in enumerate(f, 1):
        # get line until newline character
        line = line.rstrip()
        code[i] = line
print(code)

In [ ]:
%env OPENAI_API_KEY=API_KEY

In [ ]:
from openai import OpenAI
client = OpenAI()


In [ ]:
gpt_prompt = "Given the following code, generate test cases. Only respond with code that is valid Python code starting with code: from main import Product from dataclasses import is_dataclass import pytest @pytest.fixture def product(): return Product(1, 'foo', 1.0, 10) def test_constructor(product): assert product.id == 1 assert product.name == 'foo'"
print(gpt_prompt)

In [ ]:
message=[{"role": "system", "content": "You are a helpful programmer writing test.py."},{"role": "assistant", "content": gpt_prompt + "\n" + "\n".join(code.values())}]


response = client.chat.completions.create(
    model="gpt-4o",
    messages = message,
)
# print(response.choices[0].message)


In [ ]:
responseText = response.choices[0].message.content
# get rid of the first 10 characters and the last 3 characters
responseText = responseText[10:]
responseText = responseText[:-3]
responseText

In [ ]:
# if response.py already exists, delete it
import os
if os.path.exists('response.py'):
    os.remove('response.py')
# store the response in a file with name response.py
with open('response.py', 'w') as f:
    f.write(responseText)

In [ ]:
!coverage run -m pytest response.py
!coverage report -m